In [1]:
import torch
import torchvision.models as models

### 모델 가중치 저장 및 로드
* 파이토치 모델은 학습된 파라미터를 state_dict라는 내부 상태 사전에 저장
*  이러한 파라미터는 torch.save 메서드를 통해 지속될 수 있습니다.

In [2]:
model = models.vgg16(weights='IMAGENET1K_V1')
torch.save(model.state_dict(), 'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\008yo/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:09<00:00, 58.6MB/s] 


* 모델 가중치를 로드하려면 먼저 동일한 모델의 인스턴스를 생성한 다음 load_state_dict() 메서드를 사용하여 파라미터를 로드해야 함.

In [3]:
model = models.vgg16() # '가중치'를 지정하지 않습니다. 즉, 학습되지 않은 모델을 생성합니다.
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### 구조(shape)가 있는 모델 저장 및 로드
* 클래스의 구조를 모델과 함께 저장하고 싶을 수도 있는데, 이 경우에는 저장 함수에 model.state_dict()가 아닌 model을 전달하면 됩니다.

In [4]:
torch.save(model, 'model.pth')

In [5]:
# 모델 불러오기
model = torch.load('model.pth')

###  파이토치에서 일반 체크포인트 저장 및 로드하기
* 추론 또는 학습을 재개를 위해 일반 체크포인트 모델을 저장하고 로드하면 마지막으로 중단한 부분을 다시 시작할 수 있어, 유용
* 모델이 학습할 때 업데이트되는 버퍼와 파라미터가 포함되어 있으므로 옵티마이저의 state_dict도 저장하는 것이 중요
* 여러 체크포인트를 저장하려면 딕셔너리로 구성하고 torch.save()를 사용하여 딕셔너리를 직렬화해야 함
* 일반적인 파이토치 규칙은 .tar 파일 확장자를 사용하여 이러한 체크포인트를 저장하는 것
* 항목을 로드하려면 먼저 모델과 옵티마이저를 초기화한 다음 torch.load()를 사용하여 딕셔너리를 로컬로 로드

## 학습 단계
1. 데이터 로딩에 필요한 모든 라이브러리 가져오기
2. 신경망 정의 및 초기화
3. 옵티마이저 초기화
4. 일반 체크포인트 저장
5. 일반 체크포인트 로드

##### 데이터 로딩에 필요한 라이브러리 가져오기

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

##### 신경망 정의 및 초기화

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


##### 옵티마이저 초기화하기
* 모멘텀이 있는 SGD를 사용

In [8]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

##### 일반 체크포인트 저장

In [10]:
EPOCH = 5
PATH = "model.pt"
LOSS = 0.4

torch.save({
            'epoch': EPOCH,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': LOSS,
            }, PATH)

##### 일반 체크포인트 로드

In [11]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

* 추론을 실행하기 전에 model.eval()을 호출하여 드롭아웃 및 배치 정규화 레이어를 평가 모드로 설정해야 함
* 이 작업을 수행하지 않으면 일관되지 않은 추론 결과가 생성됨
* 학습을 다시 시작하려면 model.train()을 호출하여 이러한 레이어가 학습 모드에 있는지 확인